In [1]:
import pandas as pd
import numpy as np
import nltk

In [3]:
data = pd.read_csv('/content/Preprocessed_DonorsChoose_dataset.csv')

In [4]:
data.head()
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11256 entries, 0 to 11255
Data columns (total 14 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            11256 non-null  object 
 1   teacher_prefix                                11256 non-null  object 
 2   school_state                                  11256 non-null  object 
 3   project_grade_category                        11256 non-null  object 
 4   project_subject_categories                    11256 non-null  object 
 5   project_subject_subcategories                 11256 non-null  object 
 6   teacher_number_of_previously_posted_projects  11256 non-null  int64  
 7   project_is_approved                           11256 non-null  int64  
 8   price                                         11256 non-null  float64
 9   quantity                                      11256 non-null 

,0
id,0
teacher_prefix,0
school_state,0
project_grade_category,0
project_subject_categories,0
project_subject_subcategories,0
teacher_number_of_previously_posted_projects,0
project_is_approved,0
price,0
quantity,0


In [5]:
data['project_is_approved'].value_counts()
# data['project_is_approved'].value_counts(normalize=True)

,count
project_is_approved,
1,9548
0,1708


In [6]:
X = data.drop(columns=['project_is_approved'])
y = data['project_is_approved']

# Convert all columns to string and concatenate row-wise
data['combined_text'] = X.astype(str).agg(' '.join, axis=1)

data[['combined_text', 'project_is_approved']].head()

,combined_text,project_is_approved
0,p253737 mrs in grades_prek_2 literacy_language...,0
1,p258326 mr fl grades_6_8 history_civics_health...,1
2,p182444 ms az grades_6_8 health_sports health_...,0
3,p246581 mrs ky grades_prek_2 literacy_language...,1
4,p104768 mrs tx grades_prek_2 math_science math...,1


In [7]:
import pandas as pd
import string
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
def preprocess_text(text_input):
    if pd.isna(text_input):
        return []

    # 1	 Strip whitespace
    text_input = text_input.strip()

    # 2	 Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text_input = text_input.translate(translator)

    import re
    text_input = re.sub(r'[^a-zA-Z0-9 ]','',text_input)

    # 3	 Convert to lowercase
    text_input = text_input.lower()

    # 4	 Tokenize
    tokens = nltk.word_tokenize(text_input)


    # 5	 Remove stopwords
    stop_words = set(stopwords.words('english'))
    stop_word_removed = [word for word in tokens if word not in stop_words]

    # 6	 Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmitized_text = [lemmatizer.lemmatize(word) for word in stop_word_removed]

    return lemmitized_text


data['preprocess_text'] = data['combined_text'].apply(preprocess_text)

data.head()

,id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity,cleaned_titles,cleaned_essays,cleaned_summary,isdigit_summary,combined_text,preprocess_text
0,p253737,mrs,in,grades_prek_2,literacy_language,esl_literacy,0,0,154.60,23,educational support english learners home,students english learners working english seco...,students_need_opportunities_practice_beginning...,0.0,p253737 mrs in grades_prek_2 literacy_language...,"[p253737, mr, gradesprek2, literacylanguage, e..."
1,p258326,mr,fl,grades_6_8,history_civics_health_sports,civics_government_teamsports,7,1,299.00,1,wanted projector hungry learners,students arrive school eager learn polite gene...,students_need_projector_help_viewing_education...,0.0,p258326 mr fl grades_6_8 history_civics_health...,"[p258326, mr, fl, grades68, historycivicshealt..."
2,p182444,ms,az,grades_6_8,health_sports,health_wellness_teamsports,1,0,516.85,22,soccer equipment awesome middle school students,true champions not always ones win guts mia ha...,students_need_shine_guards_athletic_socks_socc...,0.0,p182444 ms az grades_6_8 health_sports health_...,"[p182444, m, az, grades68, healthsports, healt..."
3,p246581,mrs,ky,grades_prek_2,literacy_language_math_science,literacy_mathematics,4,1,232.90,4,techie kindergarteners,work unique school filled esl english second l...,students_need_engage_reading_math_way_inspire_...,0.0,p246581 mrs ky grades_prek_2 literacy_language...,"[p246581, mr, ky, gradesprek2, literacylanguag..."
4,p104768,mrs,tx,grades_prek_2,math_science,mathematics,1,1,67.98,4,interactive math tools,second grade classroom next year made around 2...,students_need_hands_practice_mathematics_fun_p...,0.0,p104768 mrs tx grades_prek_2 math_science math...,"[p104768, mr, tx, gradesprek2, mathscience, ma..."


In [9]:
data['clean_msg'] = data['preprocess_text'].apply(lambda x: ' '.join(x))
data.head()

,id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,price,quantity,cleaned_titles,cleaned_essays,cleaned_summary,isdigit_summary,combined_text,preprocess_text,clean_msg
0,p253737,mrs,in,grades_prek_2,literacy_language,esl_literacy,0,0,154.60,23,educational support english learners home,students english learners working english seco...,students_need_opportunities_practice_beginning...,0.0,p253737 mrs in grades_prek_2 literacy_language...,"[p253737, mr, gradesprek2, literacylanguage, e...",p253737 mr gradesprek2 literacylanguage esllit...
1,p258326,mr,fl,grades_6_8,history_civics_health_sports,civics_government_teamsports,7,1,299.00,1,wanted projector hungry learners,students arrive school eager learn polite gene...,students_need_projector_help_viewing_education...,0.0,p258326 mr fl grades_6_8 history_civics_health...,"[p258326, mr, fl, grades68, historycivicshealt...",p258326 mr fl grades68 historycivicshealthspor...
2,p182444,ms,az,grades_6_8,health_sports,health_wellness_teamsports,1,0,516.85,22,soccer equipment awesome middle school students,true champions not always ones win guts mia ha...,students_need_shine_guards_athletic_socks_socc...,0.0,p182444 ms az grades_6_8 health_sports health_...,"[p182444, m, az, grades68, healthsports, healt...",p182444 m az grades68 healthsports healthwelln...
3,p246581,mrs,ky,grades_prek_2,literacy_language_math_science,literacy_mathematics,4,1,232.90,4,techie kindergarteners,work unique school filled esl english second l...,students_need_engage_reading_math_way_inspire_...,0.0,p246581 mrs ky grades_prek_2 literacy_language...,"[p246581, mr, ky, gradesprek2, literacylanguag...",p246581 mr ky gradesprek2 literacylanguagemath...
4,p104768,mrs,tx,grades_prek_2,math_science,mathematics,1,1,67.98,4,interactive math tools,second grade classroom next year made around 2...,students_need_hands_practice_mathematics_fun_p...,0.0,p104768 mrs tx grades_prek_2 math_science math...,"[p104768, mr, tx, gradesprek2, mathscience, ma...",p104768 mr tx gradesprek2 mathscience mathemat...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_tfidf = tfidf_vectorizer.fit_transform(data['clean_msg'])
y = data['project_is_approved']

In [11]:
from sklearn.model_selection import train_test_split
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# TF IDF - MultinomialNB

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nb_model = MultinomialNB()
nb_model.fit(X_tfidf_train, y_tfidf_train)
y_pred = nb_model.predict(X_tfidf_test)

print(accuracy_score(y_tfidf_test,y_pred))
print(classification_report(y_tfidf_test,y_pred))
print(confusion_matrix(y_tfidf_test,y_pred))

0.8357015985790408
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       370
           1       0.84      1.00      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.42      0.50      0.46      2252
weighted avg       0.70      0.84      0.76      2252

[[   0  370]
 [   0 1882]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# TF-IDF - Logistic REGRESSION

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

lr = LogisticRegression(max_iter=1000)
lr.fit(X_tfidf_train, y_tfidf_train)

y_pred = lr.predict(X_tfidf_test)

print(accuracy_score(y_tfidf_test,y_pred))
print(classification_report(y_tfidf_test, y_pred))
print(confusion_matrix(y_tfidf_test, y_pred))

0.8357015985790408
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       370
           1       0.84      1.00      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.42      0.50      0.46      2252
weighted avg       0.70      0.84      0.76      2252

[[   0  370]
 [   0 1882]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# TF IDF - Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs= -1)
rf_model.fit(X_tfidf_train, y_tfidf_train)
y_pred = rf_model.predict(X_tfidf_test)

print(accuracy_score(y_tfidf_test,y_pred))
print(classification_report(y_tfidf_test,y_pred))
print(confusion_matrix(y_tfidf_test,y_pred))

0.8352575488454707
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       370
           1       0.84      1.00      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.42      0.50      0.46      2252
weighted avg       0.70      0.84      0.76      2252

[[   0  370]
 [   1 1881]]


# SVM

In [17]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(class_weight="balanced")

svm_model.fit(X_tfidf_train, y_tfidf_train)
y_pred = svm_model.predict(X_tfidf_test)

print(accuracy_score(y_tfidf_test,y_pred))
print(classification_report(y_tfidf_test,y_pred))
print(confusion_matrix(y_tfidf_test,y_pred))


0.8272646536412078
              precision    recall  f1-score   support

           0       0.37      0.07      0.12       370
           1       0.84      0.98      0.90      1882

    accuracy                           0.83      2252
   macro avg       0.60      0.52      0.51      2252
weighted avg       0.76      0.83      0.78      2252

[[  26  344]
 [  45 1837]]


# XGBoost

In [18]:
!pip install xgboost



In [19]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    use_label_encoder=False,
    random_state=42
)

xgb_model.fit(X_tfidf_train, y_tfidf_train)

y_pred = xgb_model.predict(X_tfidf_test)
print(accuracy_score(y_tfidf_test,y_pred))
print(classification_report(y_tfidf_test,y_pred))
print(confusion_matrix(y_tfidf_test,y_pred))

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:200: UserWarning: [05:28:46] WARNING: /__w/xgboost/xgboost/src/learner.cc:782: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0.8357015985790408
              precision    recall  f1-score   support

           0       0.50      0.04      0.07       370
           1       0.84      0.99      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.67      0.51      0.49      2252
weighted avg       0.78      0.84      0.77      2252

[[  13  357]
 [  13 1869]]


# Word2Vec

In [20]:
!pip install gensim
from gensim.models import Word2Vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 67.2 MB/s eta 0:00:00


In [21]:
import numpy as np

# First, correctly define corpus from the preprocessed text data
corpus = data['preprocess_text'].tolist()

# Then, train the Word2Vec model using this correct corpus
w2v_model = Word2Vec(sentences=corpus,
                     vector_size=50,
                     min_count = 1,
                     workers = 4, window = 3, sg=1)

def sentence_embedding(words):
  vectors = [w2v_model.wv[w] for w in words if w in w2v_model.wv]
  if len(vectors) == 0:
    return np.zeros(w2v_model.vector_size)
  else:
    return np.mean(vectors, axis=0)

# Finally, create embeddings
x_embeddings = np.array([sentence_embedding(msg) for msg in corpus])
print(x_embeddings.shape)

(11256, 50)


In [22]:
from sklearn.model_selection import train_test_split

X_w2v = x_embeddings
y = data['project_is_approved']
X_w2v_train, X_w2v_test, y_w2v_train, y_w2v_test = train_test_split(X_w2v, y, test_size=0.2, random_state=42)


# Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_w2v_train, y_w2v_train)

y_pred = lr.predict(X_w2v_test)

print(accuracy_score(y_w2v_test,y_pred))
print(classification_report(y_w2v_test, y_pred))
print(confusion_matrix(y_w2v_test, y_pred))

0.8361456483126111
              precision    recall  f1-score   support

           0       1.00      0.00      0.01       370
           1       0.84      1.00      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.92      0.50      0.46      2252
weighted avg       0.86      0.84      0.76      2252

[[   1  369]
 [   0 1882]]


# Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_w2v_train, y_w2v_train)
y_pred = rf_model.predict(X_w2v_test)

print(accuracy_score(y_w2v_test,y_pred))
print(classification_report(y_w2v_test,y_pred))
print(confusion_matrix(y_w2v_test,y_pred))

0.8352575488454707
              precision    recall  f1-score   support

           0       0.33      0.00      0.01       370
           1       0.84      1.00      0.91      1882

    accuracy                           0.84      2252
   macro avg       0.58      0.50      0.46      2252
weighted avg       0.75      0.84      0.76      2252

[[   1  369]
 [   2 1880]]


#SVM

In [25]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC(class_weight="balanced")

svm_model.fit(X_w2v_train, y_w2v_train)
y_pred = svm_model.predict(X_w2v_test)

print(accuracy_score(y_w2v_test,y_pred))
print(classification_report(y_w2v_test,y_pred))
print(confusion_matrix(y_w2v_test,y_pred))

0.6327708703374778
              precision    recall  f1-score   support

           0       0.26      0.66      0.37       370
           1       0.90      0.63      0.74      1882

    accuracy                           0.63      2252
   macro avg       0.58      0.64      0.56      2252
weighted avg       0.80      0.63      0.68      2252

[[ 245  125]
 [ 702 1180]]


In [26]:
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

# Convert sparse TF-IDF matrix to dense
X_tfidf_train_dense = X_tfidf_train.toarray()

# TF-IDF Branch

input_tfidf = Input(shape=(X_tfidf_train_dense.shape[1],))

x1 = Dense(512, activation='relu',
           kernel_initializer='he_uniform',
           kernel_regularizer=regularizers.l2(0.001))(input_tfidf)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

x1 = Dense(256, activation='relu',
           kernel_initializer='he_uniform')(x1)
x1 = Dropout(0.4)(x1)

# Word2Vec Branch

input_w2v = Input(shape=(X_w2v_train.shape[1],))

x2 = Dense(256, activation='relu',
           kernel_initializer='glorot_uniform')(input_w2v)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.4)(x2)

x2 = Dense(128, activation='relu')(x2)


# Merge Branches

merged = concatenate([x1, x2])

x = Dense(256, activation='relu',
          kernel_initializer='he_uniform')(merged)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)

# Output layer (Binary Classification)
output = Dense(1, activation='sigmoid')(x)

# Build Model
model = Model(inputs=[input_tfidf, input_w2v], outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 634766)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │ 325,000,7… │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     13,056 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 384)       │          0 │ dropout_1[0][0],  │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     98,560 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256)       │      1,024 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128)       │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │        129 │ dropout_4[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 325,313,665 (1.21 GB)

 Trainable params: 325,311,617 (1.21 GB)

 Non-trainable params: 2,048 (8.00 KB)